In [1]:
pip install pandas sqlalchemy psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [ ]:
"""
Postgres + SQLAlchemy + Pandas demo
Run this in VS Code (Python 3.10+ recommended).
"""

import pandas as pd
from sqlalchemy import create_engine, inspect, text
from sqlalchemy.exc import ProgrammingError
from getpass import getpass
import urllib.parse


def main():
    # Step 1: DB credentials
    host = "ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech"
    port = "5432"
    database = "neondb"
    user = "neondb_owner"
    password = getpass("Enter DB password: npg_CeS9fJg2azZD")  # safer than hardcoding

    # Step 2: Build engine
    password_quoted = urllib.parse.quote_plus(password)
    conn_str = f"postgresql+psycopg2://{user}:{password_quoted}@{host}:{port}/{database}"
    engine = create_engine(conn_str, pool_pre_ping=True)

    # Step 3: Test connection
    try:
        with engine.connect() as conn:
            result = conn.execute(text("SELECT 1"))
            print("✅ Connection OK, SELECT 1 returned:", result.scalar())
    except Exception as e:
        print("❌ Connection failed:", e)
        return

    inspector = inspect(engine)

    # Step 4: List all tables
    tables = inspector.get_table_names()
    print("\n📂 Tables found:", tables)

    # Step 5: Inspect `users` table
    if "users" in tables:
        cols = inspector.get_columns("users")
        df_cols = pd.DataFrame(cols)
        print("\n📑 Users table columns:")
        print(df_cols[["name", "type"]])
    else:
        print("\n⚠️ No 'users' table found.")
        return

    # Step 6: Look for datetime-like columns
    datetime_cols = [
        c["name"]
        for c in cols
        if ("timestamp" in str(c["type"]).lower())
        or ("date" in str(c["type"]).lower())
    ]
    print("\n⏰ Candidate datetime columns:", datetime_cols)

    # Step 7: Run query
    try:
        if datetime_cols:
            dt_col = datetime_cols[0]  # pick first candidate
            print(f"\n▶ Using datetime column '{dt_col}' in WHERE clause.")
            sql = text(f"""
                SELECT id, name, {dt_col}
                FROM users
                WHERE {dt_col} >= :date_limit
                LIMIT 200
            """)
            df = pd.read_sql_query(sql, engine, params={"date_limit": "2025-01-01"})
        else:
            print("\n▶ No datetime column found, selecting only id + name.")
            sql = text("SELECT id, name FROM users LIMIT 200")
            df = pd.read_sql_query(sql, engine)

        print("\n✅ Rows loaded:", len(df))
        print(df.head())
    except ProgrammingError as pe:
        print("\n⚠️ SQL error:", pe)
    finally:
        engine.dispose()
        print("\n🔌 Connection closed.")


if __name__ == "__main__":
    main()